In [1]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import RandomZoom, RandomContrast, RandomRotation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
from PIL import Image
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [3]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input 
)

In [4]:
flowers_directory=Path('flowers')

In [5]:
flowers_directory

WindowsPath('flowers')

In [6]:
list(flowers_directory.glob('*'))

[WindowsPath('flowers/daisy'),
 WindowsPath('flowers/dandelion'),
 WindowsPath('flowers/rose'),
 WindowsPath('flowers/sunflower'),
 WindowsPath('flowers/tulip')]

In [7]:
len(list(flowers_directory.glob('rose/*')))

784

In [8]:
flower_img_dict={
                 'daisy':list(flowers_directory.glob('daisy/*')),
                 'dandelion':list(flowers_directory.glob('dandelion/*')),
                 'rose':list(flowers_directory.glob('rose/*')),
                 'sunflower':list(flowers_directory.glob('sunflower/*')),
                 'tulip':list(flowers_directory.glob('tulip/*'))
}

In [9]:
flower_img_labels={
                 'daisy':0,
                 'dandelion':1,
                 'rose':2,
                 'sunflower':3,
                 'tulip':4  
}
                    

flower_name refers to the key of the dictionary, which represents the name of the flower (like 'daisy', 'dandelion', etc.).

image refers to the value associated with that key, which is a list of image paths for that particular flower.

In [10]:
x=[]
y=[]
accepted_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
for flower_name,image in flower_img_dict.items():
    print(flower_name)
    print(len(image))
    for img in image:
        # Skip non-image files (e.g., folders or checkpoint files)
        if '.ipynb_checkpoints' in str(img):
            continue
        if not img.suffix.lower() in accepted_extensions:
            continue  # Skip this file and move to the next one
        flower_img=cv2.imread(str(img))
        # Check if the image was loaded successfully
        if flower_img is None:
            continue  # Skip this image and continue to the next one
        resize_img=cv2.resize(flower_img,(224,224))
        x.append(resize_img)
        y.append(flower_img_labels[flower_name])      


daisy
765
dandelion
1052
rose
784
sunflower
733
tulip
984


In [11]:
y[:5]

[0, 0, 0, 0, 0]

In [12]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
x_train = np.array(x_train, dtype='int32')
x_test = np.array(x_test, dtype='int32')

In [14]:
x_train=x_train/255
x_test=x_test/255

In [15]:
train_generator = datagen.flow(x_train, y_train, batch_size=32, shuffle=True, seed=42)
test_generator = datagen.flow(x_test, y_test, batch_size=32, shuffle=False, seed=42)

In [16]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [17]:
# Load MobileNetV2 without the top classification layer
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Unfreeze the last few layers of the base model
base_model.trainable = True
for layer in base_model.layers[:-10]:  # Unfreeze the last 10 layers
    layer.trainable = False


# Build the model
model = Sequential([
    base_model,
    MaxPooling2D(),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')  # Adjust number of classes
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [18]:
#7. Train the Model Using the Data Generators:

#Train the model using your training and testing data generators:​

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    steps_per_epoch=len(x_train) // 32,
    validation_steps=len(x_test) // 32
)

C:\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.2266 - loss: 11.8658 - val_accuracy: 0.2188 - val_loss: 135.8250
Epoch 2/10
  1/107 ━━━━━━━━━━━━━━━━━━━━ 1:30 857ms/step - accuracy: 0.2188 - loss: 1.6506

C:\Anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


107/107 ━━━━━━━━━━━━━━━━━━━━ 27s 244ms/step - accuracy: 0.2188 - loss: 1.6506 - val_accuracy: 0.2188 - val_loss: 135.4723
Epoch 3/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.2672 - loss: 1.5979 - val_accuracy: 0.2188 - val_loss: 126.8079
Epoch 4/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 25s 230ms/step - accuracy: 0.3750 - loss: 1.5479 - val_accuracy: 0.2188 - val_loss: 126.8273
Epoch 5/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.2992 - loss: 1.5728 - val_accuracy: 0.2188 - val_loss: 109.0525
Epoch 6/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 28s 253ms/step - accuracy: 0.1875 - loss: 1.6046 - val_accuracy: 0.2188 - val_loss: 108.7263
Epoch 7/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.2924 - loss: 1.5663 - val_accuracy: 0.2188 - val_loss: 85.4883
Epoch 8/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 27s 248ms/step - accuracy: 0.3438 - loss: 1.5404 - val_accuracy: 0.2188 - val_loss: 85.3439
Epoch 9/10
107/107 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.3116 - loss: 1.5476 -